In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2023-10-03 11:18:42.046834: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-03 11:18:42.109836: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-03 11:18:42.111622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 11:18:43.593015: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def read_csv_files_from_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .csv extension
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [3]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [13]:
results = read_csv_files_from_folder('results/')
results.replace({'ftl_mlp_initializer': 'Fair Transition Loss', 'adversarial_debiasing_initializer': 'Adversarial Debiasing', 'gerry_fair_classifier_initializer': 'Gerry Fair Classifier', 'prejudice_remover_initializer': 'Prejudice Remover', 'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [14]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

In [15]:
display(results)

,Unnamed: 0,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,3,German Credit,Prejudice Remover,mcc_parity,-0.077859,0.545000,0.000000,0.070884,0.077859,0.056402,0.000000,0.077859,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,7,German Credit,Prejudice Remover,mcc_odds,-0.129086,0.620000,0.000000,0.129086,0.161000,0.084427,0.000000,0.129086,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,11,German Credit,Prejudice Remover,mcc_opportunity,-0.053052,0.575000,0.000000,0.115271,0.135058,0.053052,0.000000,0.053052,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,15,German Credit,Prejudice Remover,acc_parity,0.392316,0.555000,0.000000,0.159695,0.162684,0.195652,0.555000,0.162684,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,19,German Credit,Prejudice Remover,acc_odds,0.469445,0.555000,0.000000,0.085555,0.021354,0.117521,0.555000,0.085555,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,1734,Bank Marketing,Fair Transition Loss,mcc_odds,0.512368,0.892752,0.567577,0.055209,0.064039,0.096869,0.567577,0.055209,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
1571,1739,Bank Marketing,Fair Transition Loss,mcc_opportunity,0.439395,0.787963,0.501721,0.423663,0.696893,0.062327,0.501721,0.062327,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
1572,1744,Bank Marketing,Fair Transition Loss,acc_parity,0.885779,0.893244,0.378632,0.047285,0.007465,0.079024,0.893244,0.007465,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
1573,1749,Bank Marketing,Fair Transition Loss,acc_odds,0.823758,0.895048,0.393785,0.071289,0.012262,0.137643,0.895048,0.071289,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)


In [19]:
datasets = results['dataset'].unique().tolist().sort()
datasets

In [17]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [18]:
methods = ['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']
methods

['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']

In [12]:
if os.path.exists('multi_aso_data_list.json'):
    with open('multi_aso_data_list.json') as file:
        multi_aso_data_list = json.load(file)
else:    
    multi_aso_data_list = []
    for d in datasets:
        multi_aso_data = []
        for f in fitness_rules:
            methods_results = []
            for m in methods:
                r = results.loc[ (results['dataset'] == d) &
                                     (results['fitness_rule'] == f) &
                                     (results['method'] == m) ]\
                            .fitness.tolist()
                if len(r) == 0:
                    r = [-1]
                methods_results.append(r)
            min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
            multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
    with open('multi_aso_data_list.json', 'w') as file:
        json.dump(multi_aso_data_list, file)

In [13]:
multi_aso_data_list

[{'fitness_rule': 'mcc_parity',
  'dataset': 'Adult Income',
  'min_eps': [[1.0,
    0.9988805908520648,
    0.9979750223790821,
    0.995877444722284,
    0.0],
   [0.004620261792774651, 1.0, 0.1528664058142362, 1.0, 0.0],
   [0.004940540275507269, 1.0, 1.0, 1.0, 0.0],
   [0.00595911008417298, 1.0, 0.1050132284790311, 1.0, 0.0],
   [1.0, 1.0, 1.0, 1.0, 1.0]]},
 {'fitness_rule': 'mcc_odds',
  'dataset': 'Adult Income',
  'min_eps': [[1.0, 1.0, 1.0, 1.0, 0.0],
   [0.027617628263128934, 1.0, 0.25458170757222737, 0.42157426878629345, 0.0],
   [0.251460472570234, 1.0, 1.0, 0.7790530472236021, 0.0],
   [0.8744194609335025, 1.0, 1.0, 1.0, 0.04857425621287411],
   [1.0, 1.0, 1.0, 1.0, 1.0]]},
 {'fitness_rule': 'mcc_opportunity',
  'dataset': 'Adult Income',
  'min_eps': [[1.0,
    0.9976536340274126,
    0.9789694086170047,
    0.8910534937566084,
    0.0],
   [0.008614325158317561,
    1.0,
    0.0032188112253179217,
    0.058227525767044014,
    0.0],
   [1.0, 1.0, 1.0, 0.9400209938187005, 

In [18]:
aso_df_resume = []
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['method'] = methods
    aso_df['dataset'] = dataset
    aso_df['fitness_rule'] = fitness_rule
    aso_df = aso_df[aso_df['method'] == 'Fair Transition Loss' ]
    aso_df = aso_df.drop(['Fair Transition Loss'], axis=1)
    aso_df = aso_df.drop(['method'], axis=1)
    aso_df_resume.append(aso_df)


print('Significance Testing')
significance = pd.concat(aso_df_resume)
significance = significance.set_index(['fitness_rule', 'dataset'])
significance = significance.sort_values(by=['fitness_rule', 'dataset'])
display(significance)
significance.to_latex('significance_resume.tex')

Significance Testing


Standard MLP (baseline)  \
fitness_rule    dataset                                      
acc_odds        Adult Income                      0.009395   
                Bank Market                       0.775530   
                Compas Recidivism                 0.006612   
                German Credit                     1.000000   
acc_opportunity Adult Income                      0.898703   
                Bank Market                       1.000000   
                Compas Recidivism                 0.009027   
                German Credit                     0.014195   
acc_parity      Adult Income                      0.006226   
                Bank Market                       0.235047   
                Compas Recidivism                 0.002008   
                German Credit                     1.000000   
mcc_odds        Adult Income                      0.027618   
                Bank Market                       0.471688   
                Compas Recidivism                 0.005317   
                German Credit                     1.000000   
mcc_opportunity Adult Income                      0.008614   
                Bank Market                       0.820683   
                Compas Recidivism                 0.004837   
                German Credit                     1.000000   
mcc_parity      Adult Income                      0.004620   
                Bank Market                       0.008002   
                Compas Recidivism                 0.005095   
                German Credit                     0.362420   

                                   Adversarial Debiasing  Prejudice Remover  \
fitness_rule    dataset                                                       
acc_odds        Adult Income                    0.225552           0.170713   
                Bank Market                     0.386310           0.814048   
                Compas Recidivism               0.471443           0.000000   
                German Credit                   0.429356           0.969500   
acc_opportunity Adult Income                    0.959536           1.000000   
                Bank Market                     0.369527           0.819028   
                Compas Recidivism               0.792935           0.000000   
                German Credit                   0.082456           0.000000   
acc_parity      Adult Income                    0.252930           0.305378   
                Bank Market                     1.000000           1.000000   
                Compas Recidivism               1.000000           0.107223   
                German Credit                   0.469207           1.000000   
mcc_odds        Adult Income                    0.254582           0.421574   
                Bank Market                     0.189642           0.108138   
                Compas Recidivism               0.582610           0.000000   
                German Credit                   1.000000           1.000000   
mcc_opportunity Adult Income                    0.003219           0.058228   
                Bank Market                     0.180293           0.236480   
                Compas Recidivism               1.000000           0.000000   
                German Credit                   1.000000           1.000000   
mcc_parity      Adult Income                    0.152866           1.000000   
                Bank Market                     0.000000           0.000000   
                Compas Recidivism               0.211472           0.000000   
                German Credit                   1.000000           0.475996   

                                   Gerry Fair Classifier  
fitness_rule    dataset                                   
acc_odds        Adult Income                    0.000000  
                Bank Market                     1.000000  
                Compas Recidivism               0.001646  
                German Credit                   0.000000  
acc_opportuni

In [19]:
grouped_results = results\
    .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'mean')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'mean')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'mean')]:.2f})$", axis=1)
grouped_results

fitness  \
                                                                       mean   
Fitness Rule Abvr       dataset           method                              
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing    0.200070   
                                          Fair Transition Loss    -0.034435   
                                          Prejudice Remover       -0.082559   
                                          Standard MLP (baseline) -0.090370   
                        Compas Recidivism Fair Transition Loss     0.219648   
...                                                                     ...   
Max(Acc - Eq. Odds)     Adult Income      Prejudice Remover        0.809534   
                                          Fair Transition Loss     0.787013   
                                          Adversarial Debiasing    0.756054   
                                          Standard MLP (baseline)  0.752260   
                                          Gerry Fair Classifier    0.655568   

                                                                             \
                                                                        std   
Fitness Rule Abvr       dataset           method                              
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing    0.172675   
                                          Fair Transition Loss     0.073668   
                                          Prejudice Remover        0.050746   
                                          Standard MLP (baseline)  0.061469   
                        Compas Recidivism Fair Transition Loss     0.060505   
...                                                                     ...   
Max(Acc - Eq. Odds)     Adult Income      Prejudice Remover        0.022701   
                                          Fair Transition Loss     0.083934   
                                          Adversarial Debiasing    0.028345   
                                          Standard MLP (baseline)  0.041790   
                                          Gerry Fair Classifier    0.185910   

                                                                  Performance  \
                                                                         mean   
Fitness Rule Abvr       dataset           method                                
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing      0.367673   
                                          Fair Transition Loss       0.000000   
                                          Prejudice Remover          0.000000   
                                          Standard MLP (baseline)    0.000000   
                        Compas Recidivism Fair Transition Loss       0.276319   
...                                                                       ...   
Max(Acc - Eq. Odds)     Adult Income      Prejudice Remover          0.845788   
                                          Fair Transition Loss       0.825826   
                                          Adversarial Debiasing      0.847730   
                                          Standard MLP (baseline)    0.849309   
                                          Gerry Fair Classifier      0.720083   

                                                                             \
                                                                        std   
Fitness Rule Abvr       dataset           method                              
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing    0.056485   
                                          Fair Transition Loss     0.000000   
                                          Prejudice Remover        0.000000   
                                          Standard MLP (baseline)  0.000000   
                        Compas Recidivism Fair Transition Loss     0.027969   
...                                                                     ...   
Max(Acc 

In [20]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
grouped_results[selected_columns].to_latex('grouped_results.tex')